# introduce

- public_api提供公共Api,无需认证/授权

In [3]:
import ccxt
exchange_ins = ccxt.binance()
exchange_ins

ccxt.binance()

# Markets

- In ccxt：
  - every exchange offers multiple markets(coin) within itself.
  - Each market is defined by two or more currencies(USDT,USDC).


## Load all


In [4]:
# 注意 此方法会产生本地缓存,第二次调用使用缓存
market = exchange_ins.load_markets()

In [ ]:
# 无缓存
market = exchange_ins.load_markets(reload=True)

## Get One


In [ ]:
btc_market = exchange_ins.markets["BTC/USDT"]
btc_market

{'id': 'BTCUSDT',
 'lowercaseId': 'btcusdt',
 'symbol': 'BTC/USDT',
 'base': 'BTC',
 'quote': 'USDT',
 'settle': None,
 'baseId': 'BTC',
 'quoteId': 'USDT',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'subType': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 1e-05,
  'price': 0.01,
  'cost': None,
  'base': 1e-08,
  'quote': 1e-08},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 1e-05, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 5.0, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 108.87919041}},
 'created': None,
 'info': {'symbol': 'BTCUSDT',
  'status': 'TRADING',
  'baseAsset': 'BTC',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  '

In [ ]:
eth_market = exchange_ins.markets["ETH/USDT"]
eth_market

{'id': 'ETHUSDT',
 'lowercaseId': 'ethusdt',
 'symbol': 'ETH/USDT',
 'base': 'ETH',
 'quote': 'USDT',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'USDT',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'subType': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 2, 'cost': None, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 5.0, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 2382.977595}},
 'created': None,
 'info': {'symbol': 'ETHUSDT',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  'quotePrecision': '8',
 

## Advanced Filter


In [ ]:
from typing import Any


def get_markets(
    spot_only: bool = False,
    margin_only: bool = False,
    futures_only: bool = False,
    tradable_only: bool = True,
    active_only: bool = False,
) -> dict[str, Any]:
    """
    Return exchange ccxt markets, filtered out by base currency and quote currency
    if this was requested in parameters.
    """
    markets = exchange_ins.load_markets()
    if not markets:
        raise RuntimeError("Markets were not loaded.")

    if spot_only:
        markets = {k: v for k, v in markets.items() if v.get("spot", False) is True}
    if margin_only:
        markets = {k: v for k, v in markets.items() if v.get("margin", False) is True}
    if futures_only:
        #  the contract is a linear contract (settled in quote currency)
        markets = {k: v for k, v in markets.items() if v.get("linear", False) is True}
    if active_only:
        markets = {
            k: v for k, v in markets.items() if v.get("active", True) is not False
        }

    return markets

In [ ]:
spot = get_markets(spot_only=True)
spot.get("ETH/USDT:USDT")  # 获取合约,返回空

In [ ]:
spot.get("ETH/USDT")  # 获取现货ok

{'id': 'ETHUSDT',
 'lowercaseId': 'ethusdt',
 'symbol': 'ETH/USDT',
 'base': 'ETH',
 'quote': 'USDT',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'USDT',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'subType': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 2, 'cost': None, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 5.0, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 2382.977595}},
 'created': None,
 'info': {'symbol': 'ETHUSDT',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  'quotePrecision': '8',
 

# ohlcv


In [ ]:
from datetime import datetime, timedelta

# 一天前
time_stamp = int((datetime.now() - timedelta(days=1)).timestamp()) * 1000
print(time_stamp)
ohlcv = exchange_ins.fetch_ohlcv(
    symbol="BTC/USDT:USDT", timeframe="5m", since=time_stamp, limit=300, params={}
)
print(type(ohlcv))  # <class 'list'>

1720690821000
<class 'list'>


In [ ]:
import pandas as pd

DEFAULT_DATAFRAME_COLUMNS = ["date", "open", "high", "low", "close", "volume"]
df = pd.DataFrame(ohlcv, columns=DEFAULT_DATAFRAME_COLUMNS)
df["date"] = pd.to_datetime(df["date"], unit="ms", utc=True)
df

,date,open,high,low,close,volume
0,2024-07-11 09:45:00+00:00,58207.4,58212.8,58091.1,58173.6,468.517
1,2024-07-11 09:50:00+00:00,58173.7,58222.6,58139.3,58213.7,282.827
2,2024-07-11 09:55:00+00:00,58213.7,58293.5,58205.6,58265.7,468.394
3,2024-07-11 10:00:00+00:00,58265.8,58489.8,58239.6,58366.2,2218.911
4,2024-07-11 10:05:00+00:00,58366.2,58376.9,58213.0,58260.0,1013.294
...,...,...,...,...,...,...
283,2024-07-12 09:20:00+00:00,57153.6,57196.4,57145.5,57166.5,157.885
284,2024-07-12 09:25:00+00:00,57166.6,57184.8,57142.8,57161.6,166.331
285,2024-07-12 09:30:00+00:00,57161.9,57188.6,57081.7,57150.1,430.339
286,2024-07-12 09:35:00+00:00,57150.0,57157.1,57000.0,57087.0,1510.345


# coin

force on one coin


## long/short ratio

TODO


## order book ratio

TODO


# Ticker

行情数据


In [ ]:
btc_ticker = exchange_ins.fetch_ticker("BTC/USDT")
eth_ticker = exchange_ins.fetch_ticker("ETH/USDT")

print(btc_ticker["last"])
print(eth_ticker["last"])

63176.65
3411.46
